In [1]:
## Đọc dữ liệu từ tệp

import math
import numpy as np
with open('./data/fuel.txt') as f:
    lines = f.readlines()

x_data = []
y_data = []
lines.pop(0)

for line in lines:
    splitted = line.replace('\n', '').split(',')
    splitted.pop(0)
    splitted = list(map(float, splitted))
    fuel = 1000 * splitted[1] / splitted[5]
    dlic = 1000 * splitted[0] / splitted[5]
    logMiles = math.log2(splitted[3])
    y_data.append([fuel])
    x_data.append([splitted[-1], dlic, splitted[2], logMiles])

x_data = np.asarray(x_data)
y_data = np.asarray(y_data)

In [2]:
## Trường hợp sử dụng Numpy – Dùng thuật toán HoldHouse để khai triển QR

def qr_householder(A):
    #""" Compute QR decomposition of A using Householder reflection"""
    M = A.shape[0]
    N = A.shape[1]

    # set Q to the identity matrix
    Q = np.identity(M)

    # set R to zero matrix
    R = np.copy(A)

    for n in range(N):
        # vector to transform
        x = A[n:, n]
        k = x.shape[0]

        # compute ro=-sign(x0)||x||
        ro = -np.sign(x[0]) * np.linalg.norm(x)

        # compute the householder vector v
        e = np.zeros(k)
        e[0] = 1
        v = (1 / (x[0] - ro)) * (x - (ro * e))

        # apply v to each column of A to find R
        for i in range(N):
            R[n:, i] = R[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ R[n:, i])

        # apply v to each column of Q
        for i in range(M):
            Q[n:, i] = Q[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ Q[n:, i])

    return Q.transpose(), R

def linear_regression(x_data, y_data):
# """
# This function calculate linear regression base on x_data and y_data
# :param x_data: vector
# :param y_data: vector
# :return: w (regression estimate)
# """

# add column 1
    x_bars = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)

    Q, R = qr_householder(x_bars) # QR decomposition
    R_pinv = np.linalg.pinv(R) # calculate inverse matrix of R
    A = np.dot(R_pinv, Q.T) # apply formula

    return np.dot(A, y_data)

In [3]:
w = linear_regression(x_data, y_data) # get result
w = w.T.tolist()
line = ['Intercept', 'Tax', "Dlic", "Income", 'LogMiles']
res = list(zip(line, w[0]))
for o in res:
    print("{: >20}: {: >10}".format(*o))

           Intercept: 154.19284457730794
                 Tax: -4.227983208329624
                Dlic: 0.4718712134419837
              Income: -0.0061353309704178065
            LogMiles: 18.545274506048024


In [9]:
# Trường hợp sử dụng thư viện Scikit-Learn
from sklearn import datasets, linear_model
# Load training data here and assign to Xbar (obs. Data) and y (label)
# fit the model by Linear Regression
regr = linear_model.LinearRegression(fit_intercept=False)
# fit_intercept = False for calculating the bias
x_bars = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)
regr.fit(x_bars, y_data) # train

# <model_name>.predict(X_input) # predict
# <model_name>.coef # weights of model
# Dự đoán đầu ra cho dữ liệu mới (nếu cần)
# predictions = regr.predict(new_data)

# Trọng số của mô hình
coefficients = regr.coef_

# In ra trọng số của mô hình
print("Trọng số của mô hình:")
for i, coef in enumerate(coefficients[0]):
    print(f"{line[i]}: {coef}")

Trọng số của mô hình:
Intercept: 154.19284457727477
Tax: -4.227983208329612
Dlic: 0.47187121344198957
Income: -0.0061353309704177406
LogMiles: 18.54527450604802
